In [ ]:
import numpy



class FluxLimiter:
    def __init__(self, uminus, ucenter, uplus, flux_limiter):
        self.uminus = uminus
        self.ucenter = ucenter
        self.uplus = uplus

        self.R = numpy.array(
            [(e - c) / (c - w) if c != w else 0 for w, c, e in zip(self.uminus, self.ucenter, self.uplus)]
        )

        self.flux_limiter = flux_limiter

    @staticmethod
    def superbee(R):
        return numpy.minimum(numpy.maximum(1, R), 2, 2 * R)

    @staticmethod
    def vanLeer(R):
        return 2 * R / (R + 1)

    @staticmethod
    def vanAlbada(R):
        return (R * R + R) / (R * R + 1)

    def right_interface_left_limit(self):
        return self.ucenter + 0.5 * self.flux_limiter(self.R) * (self.uplus - self.uminus)

    def left_interface_right_limit(self):
        return self.ucenter - 0.5 * self.flux_limiter(self.R) * (self.uplus - self.uminus)


class RiemannSolver:
    @staticmethod
    def intercell_flux_lax_friedrichs(F, U_L, U_R, S_L, S_R):
        return 0.5 * (F(U_L) + F(U_R)) - 0.5 * max(abs(S_L), abs(S_R)) * (U_R - U_L)

    @staticmethod
    def intercell_flux_hll(F, U_L, U_R, S_L, S_R):
        if 0 <= S_L:
            return F(U_L)
        elif S_L <= 0 <= S_R:
            return (S_R * F(U_L) - S_L * F(U_R) + S_L * S_R * (U_R - U_L)) / (S_R - S_L)
        elif S_R <= 0:
            return F(U_R)
        else:
            raise ValueError()

    @staticmethod
    def intercell_flux(F, U_L, U_R, S_L, S_R):
        return RiemannSolver.intercell_flux_hll(F, U_L, U_R, S_L, S_R)


1D Shallow Water Equations:

$$
\begin{align}
    z_w &= z_b + h\\
    \partial_t(h) + \partial_0(h u_0) &= 0\\
    \partial_t(h u_0) + \partial_0(h u_0 u_0) + \partial_0(\frac{1}{2} g h^2) + g h\partial_0(z_b) + C_f |u| u_0 &= 0\\
    \partial_t(g) &= 0\\
    \partial_t(C_f) &= 0\\
\end{align}
$$

$$
Q = 
\left[\begin{matrix}
b\\
w\\
(w - b) u_0\\
g\\
\partial_0(z_b)\\
C_f\\
\end{matrix}\right]\\
$$

$$
\begin{array}{lllllllll}
\partial_t&[
    &Q_0 
    &Q_1
    &Q_2
    &Q_3
    &Q_4 
    &Q_5
&]^\top\\
\partial_0&[
    &Q_0 Q_2 (Q_1 - Q_0)^{-1}
    &Q_1 Q_2 (Q_1 - Q_0)^{-1}
    &Q_2 Q_2 (Q_1 - Q_0)^{-1} + \frac{1}{2}Q_3 (Q_1 - Q_0)^2 
    &0 
    &0 
    &0
&]^\top\\
&[
&0
&0
&Q_3 (Q_1 - Q_0) Q_4 + Q_5 \left\|Q_2 (Q_1 - Q_0)^{-1}\right\| Q_2 (Q_1 - Q_0)^{-1}
&0
&0
&0
&]^\top
\end{array}\\
= \partial_t Q + \partial_0 F(Q) + S(Q) = 0\\
$$

$$
\partial_Q F
=
\left(
                  \begin{array}{cccccc}
                   \frac{Q_1 Q_2}{\left(Q_0-Q_1\right){}^2} & -\frac{Q_0 Q_2}{\left(Q_0-Q_1\right){}^2} &
                     \frac{Q_0}{Q_1-Q_0} & 0 & 0 & 0 \\
                   \frac{Q_1 Q_2}{\left(Q_0-Q_1\right){}^2} & -\frac{Q_0 Q_2}{\left(Q_0-Q_1\right){}^2} &
                     \frac{Q_1}{Q_1-Q_0} & 0 & 0 & 0 \\
                   \frac{Q_2{}^2}{\left(Q_0-Q_1\right){}^2}+\left(Q_0-Q_1\right) Q_5 & \left(Q_1-Q_0\right)
                     Q_5-\frac{Q_2{}^2}{\left(Q_0-Q_1\right){}^2} & \frac{2 Q_2}{Q_1-Q_0} & \frac{1}{2}
                     \left(Q_0-Q_1\right){}^2 & 0 & 0 \\
                   0 & 0 & 0 & 0 & 0 & 0 \\
                   0 & 0 & 0 & 0 & 0 & 0 \\
                   0 & 0 & 0 & 0 & 0 & 0 \\
                  \end{array}
                  \right)
$$

$$
\left(
                  \begin{array}{cccccc}
                   0 & 0 & 0 & -\frac{Q_2}{Q_0-Q_1} & -\frac{Q_0 Q_2-Q_1 Q_2+\sqrt{-\left(Q_0-Q_1\right){}^5
                     Q_5}}{\left(Q_0-Q_1\right){}^2} & \frac{-Q_0 Q_2+Q_1 Q_2+\sqrt{-\left(Q_0-Q_1\right){}^5
                     Q_5}}{\left(Q_0-Q_1\right){}^2} \\
                   \{0,0,0,0,0,1\} & \{0,0,0,0,1,0\} & \left\{-\frac{Q_0 \left(Q_0-Q_1\right){}^3}{2 \left(Q_5
                     \left(Q_0-Q_1\right){}^3+Q_2{}^2\right)},-\frac{\left(Q_0-Q_1\right){}^3 Q_1}{2 \left(Q_5
                     \left(Q_0-Q_1\right){}^3+Q_2{}^2\right)},0,1,0,0\right\} & \{1,1,0,0,0,0\} & \left\{\frac{Q_0
                     \sqrt{-\left(Q_0-Q_1\right){}^5 Q_5}}{-Q_5 Q_0{}^4+4 Q_1 Q_5 Q_0{}^3-6 Q_1{}^2 Q_5 Q_0{}^2+4
                     Q_1{}^3 Q_5 Q_0-Q_1{}^4 Q_5+Q_2 \sqrt{\left(Q_1-Q_0\right){}^5 Q_5}},\frac{Q_1
                     \sqrt{\left(Q_1-Q_0\right){}^5 Q_5}}{-Q_5 Q_0{}^4+4 Q_1 Q_5 Q_0{}^3-6 Q_1{}^2 Q_5 Q_0{}^2+4 Q_1{}^3
                     Q_5 Q_0-Q_1{}^4 Q_5+Q_2 \sqrt{\left(Q_1-Q_0\right){}^5 Q_5}},1,0,0,0\right\} & \left\{\frac{Q_0
                     \sqrt{-\left(Q_0-Q_1\right){}^5 Q_5}}{Q_5 Q_0{}^4-4 Q_1 Q_5 Q_0{}^3+6 Q_1{}^2 Q_5 Q_0{}^2-4 Q_1{}^3
                     Q_5 Q_0+Q_1{}^4 Q_5+Q_2 \sqrt{\left(Q_1-Q_0\right){}^5 Q_5}},\frac{Q_1
                     \sqrt{\left(Q_1-Q_0\right){}^5 Q_5}}{Q_5 Q_0{}^4-4 Q_1 Q_5 Q_0{}^3+6 Q_1{}^2 Q_5 Q_0{}^2-4 Q_1{}^3
                     Q_5 Q_0+Q_1{}^4 Q_5+Q_2 \sqrt{\left(Q_1-Q_0\right){}^5 Q_5}},1,0,0,0\right\} \\
                  \end{array}
                  \right)
$$